### https://data.police.uk/docs/method/crime-street/

In [1]:
import requests
import csv
import praw
import requests
import pandas as pd
import re
import time
import spacy
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
import folium
from IPython.display import display, HTML
from nltk import ngrams
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Bhupendra
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# Define the API endpoint and parameters
url = 'https://data.police.uk/api/crimes-street/all-crime'
params = {
    'lat': '52.629729',  # replace with desired latitude
    'lng': '-1.131592',  # replace with desired longitude
    'date': '2023-05'    # replace with desired date
}

# Make the API request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    try:
        data = response.json()
    except requests.JSONDecodeError:
        print("Error: Unable to decode JSON response")
        print("Raw response content:", response.text)
        data = None
else:
    print(f"Error: Request failed with status code {response.status_code}")
    print("Raw response content:", response.text)
    data = None

if data:
    # Define the CSV file name
    csv_file = 'crime_data.csv'

    # Specify the header
    header = ["category", "location_type", "latitude", "longitude", "street_id", "street_name", "context", "outcome_status", "persistent_id", "id", "location_subtype", "month"]

    # Write the data to CSV
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)
        for crime in data:
            writer.writerow([
                crime.get("category", ""),
                crime.get("location_type", ""),
                crime["location"].get("latitude", ""),
                crime["location"].get("longitude", ""),
                crime["location"]["street"].get("id", ""),
                crime["location"]["street"].get("name", ""),
                crime.get("context", ""),
                crime.get("outcome_status", {}).get("category", "") if crime.get("outcome_status") else "",
                crime.get("persistent_id", ""),
                crime.get("id", ""),
                crime.get("location_subtype", ""),
                crime.get("month", "")
            ])

    print(f"Data saved to {csv_file}")

Data saved to crime_data.csv


In [3]:
df = pd.read_csv('crime_data.csv')

In [4]:
df.head()

,category,location_type,latitude,longitude,street_id,street_name,context,outcome_status,persistent_id,id,location_subtype,month
0,anti-social-behaviour,Force,52.626216,-1.147215,1737573,On or near Western Road,NaN,NaN,NaN,110254485,NaN,2023-05
1,anti-social-behaviour,Force,52.637146,-1.149381,1737432,On or near Vaughan Street,NaN,NaN,NaN,110254489,NaN,2023-05
2,anti-social-behaviour,Force,52.633880,-1.114617,1739287,On or near Mensa Close,NaN,NaN,NaN,110254490,NaN,2023-05
3,anti-social-behaviour,Force,52.627299,-1.148966,1737468,On or near Norman Street,NaN,NaN,NaN,110254500,NaN,2023-05
4,anti-social-behaviour,Force,52.629134,-1.129281,1738570,On or near Museum Square,NaN,NaN,NaN,110254509,NaN,2023-05


In [5]:
import folium

In [6]:
# Remove rows with any NaN values
df = df.dropna(subset=['latitude', 'longitude'])

# Create a map centered around the average location of the crimes
map_center = [df['latitude'].mean(), df['longitude'].mean()]
crime_map = folium.Map(location=map_center, zoom_start=13)

# Add points to the map
for index, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"Category: {row['category']}\nStreet: {row['street_name']}\nOutcome: {row['outcome_status']}",
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(crime_map)

# Save the map to an HTML file
crime_map.save('crime_map.html')

# Display the map in Jupyter Notebook (if using a Jupyter environment)
crime_map
